In [1]:
import os
import subprocess
import sys
import scanpy as sc
import numpy as np
import scipy
import scanpy.external as sce

In [2]:
inputs=['../../../oscb/user_storage/Benchmarks/droplet-Bladder_1751255009850/QC/droplet_Bladder_Seurat.h5ad', '../../../oscb/user_storage/Benchmarks/facs-Bladder_1751255094313/QC/facs_Bladder_Seurat.h5ad']

adatas = [sc.read_h5ad(input) for input in inputs]
adata = sc.concat(adatas, join='outer')
adata

AnnData object with n_obs × n_vars = 3878 × 3427
    obs: 'orig.ident', 'n_counts', 'n_genes', 'channel', 'tissue', 'subtissue', 'mouse.sex', 'mouse.id', 'percent.ercc', 'percent.ribo', 'free_annotation', 'cell_ontology_class', 'res.0.4', 'previous_free_annotation', 'previous_cell_ontology_class', 'cluster.ids', 'cell_ontology_id', 'pct_counts_mt', 'pct_counts_rb', 'pct_counts_hb', 'percent.plat', 'RNA_snn_res.0.5', 'seurat_clusters', 'doublet_score', 'doublet_class', 'leiden', 'louvain', 'nReads', 'plate.barcode', 'FACS.selection'
    obsm: 'X_pca', 'X_tsne', 'X_umap', 'X_umap_3D'
    layers: 'raw_counts', 'scale.data'

In [5]:
# Pseudo replicates
def create_pseudo_replicates(adata, batch_key, num):
    import random
    ads = []
    for sample in adata.obs[batch_key].unique():
        samp_cell_subset = adata[adata.obs[batch_key] == sample]
        
        samp_cell_subset.X = samp_cell_subset.layers['raw_counts'] #make sure to use raw data
        
        indices = list(samp_cell_subset.obs_names)
        random.shuffle(indices)
        indices = np.array_split(np.array(indices), num) #change number here for number of replicates deisred
        
        for i, pseudo_rep in enumerate(indices):
        
            # rep_adata = sc.AnnData(X = samp_cell_subset[indices[i]].X.sum(axis = 0),
            #                        var = samp_cell_subset[indices[i]].var[[]])
            rep_adata = samp_cell_subset[samp_cell_subset.obs_names.isin(pseudo_rep)]
            rep_adata.obs['sample'] = sample + '_' + str(i + 1)
            # rep_adata.obs['condition'] = samp_cell_subset.obs['condition'].iloc[0]
            rep_adata.obs['replicate'] = i + 1

            ads.append(rep_adata)
    ad = sc.concat(ads)
    
    return ad

In [6]:
adata = create_pseudo_replicates(adata, 'orig.ident', 3)

/tmp/ipykernel_79/3328415220.py:19: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  rep_adata.obs['sample'] = sample + '_' + str(i + 1)
/tmp/ipykernel_79/3328415220.py:19: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  rep_adata.obs['sample'] = sample + '_' + str(i + 1)
/tmp/ipykernel_79/3328415220.py:19: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  rep_adata.obs['sample'] = sample + '_' + str(i + 1)
/tmp/ipykernel_79/3328415220.py:19: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  rep_adata.obs['sample'] = sample + '_' + str(i + 1)
/tmp/ipykernel_79/3328415220.py:19: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  rep_adata.obs['sample'] = sample + '_' + str(i + 1)
/tmp/ipykernel_79/3328415220.py:19: ImplicitM

In [7]:
adata.obs

,orig.ident,n_counts,n_genes,channel,tissue,subtissue,mouse.sex,mouse.id,percent.ercc,percent.ribo,...,seurat_clusters,doublet_score,doublet_class,leiden,louvain,nReads,plate.barcode,FACS.selection,sample,replicate
10X_P4_3_AACTCTTAGTTGCAGG,10X,3864.0,609,10X_P4_3,Bladder,,M,3-M-8,0.000000,0.356385,...,3,0.250000,Singlet,6,14,NaN,NaN,NaN,10X_1,1
10X_P4_3_AAGACCTAGATCCGAG,10X,4166.0,522,10X_P4_3,Bladder,,M,3-M-8,0.000000,0.237780,...,8,0.323333,Doublet,8,11,NaN,NaN,NaN,10X_1,1
10X_P4_3_AAGTCTGAGATAGTCA,10X,1787.0,421,10X_P4_3,Bladder,,M,3-M-8,0.000000,0.225972,...,3,0.063333,Singlet,2,3,NaN,NaN,NaN,10X_1,1
10X_P4_3_ACGAGCCAGATAGCAT,10X,2313.0,403,10X_P4_3,Bladder,,M,3-M-8,0.000000,0.163284,...,6,0.083333,Singlet,7,12,NaN,NaN,NaN,10X_1,1
10X_P4_3_ACGGGCTGTCTGATCA,10X,1499.0,330,10X_P4_3,Bladder,,M,3-M-8,0.000000,0.172143,...,2,0.036667,Singlet,7,12,NaN,NaN,NaN,10X_1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
P4.MAA000487.3_10_M.1.1,Bladder,252010.0,638,NaN,Bladder,,M,3_10_M,0.007899,0.020916,...,0,0.175758,Singlet,1,0,1467200.0,MAA000487,Multiple,Bladder_3,3
P5.D041914.3_8_M.1.1,Bladder,18873.0,455,NaN,Bladder,,M,3_8_M,0.014337,0.041062,...,0,0.090909,Singlet,1,5,157227.0,D041914,Multiple,Bladder_3,3
P7.D042253.3_9_M.1.1,Bladder,49225.0,494,NaN,Bladder,,M,3_9_M,0.007055,0.023075,...,2,0.103030,Singlet,0,0,272071.0,D042253,Multiple,Bladder_3,3
P9.B000610.3_56_F.1.1,Bladder,121666.0,522,NaN,Bladder,,F,3_56_F,0.008582,0.065607,...,4,0.139394,Singlet,3,2,865291.0,B000610,Multiple,Bladder_3,3


In [10]:
'raw_counts' in adata.layers.keys()

True

In [6]:
adata.X.todense()

matrix([[  0.,   0.,   0., ...,   0.,   0.,   0.],
        [  0.,   0.,   0., ...,   0.,   1.,   0.],
        [  0.,   0.,   0., ...,   0.,   1.,   0.],
        ...,
        [  1.,   0.,   0., ...,   0.,   0.,   0.],
        [  2.,   0.,  18., ...,   0.,   0.,   0.],
        [112.,   0.,  37., ...,   0.,   0.,  17.]], dtype=float32)

In [7]:
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)

In [10]:
adata.obs

,orig.ident,n_counts,n_genes,channel,tissue,subtissue,mouse.sex,mouse.id,percent.ercc,percent.ribo,...,percent.plat,RNA_snn_res.0.5,seurat_clusters,doublet_score,doublet_class,leiden,louvain,nReads,plate.barcode,FACS.selection
10X_P4_3_AAAGTAGAGATGCCAG,10X,2294.0,404,10X_P4_3,Bladder,,M,3-M-8,0.000000,0.219609,...,0.0,6,6,0.076667,Singlet,7,12,NaN,NaN,NaN
10X_P4_3_AACCGCGTCCAACCAA,10X,4067.0,548,10X_P4_3,Bladder,,M,3-M-8,0.000000,0.176712,...,0.0,0,0,0.320000,Doublet,7,12,NaN,NaN,NaN
10X_P4_3_AACTCCCGTCGGGTCT,10X,2699.0,474,10X_P4_3,Bladder,,M,3-M-8,0.000000,0.192444,...,0.0,0,0,0.210000,Singlet,7,12,NaN,NaN,NaN
10X_P4_3_AACTCTTAGTTGCAGG,10X,3864.0,609,10X_P4_3,Bladder,,M,3-M-8,0.000000,0.356385,...,0.0,3,3,0.250000,Singlet,6,14,NaN,NaN,NaN
10X_P4_3_AACTCTTTCATAACCG,10X,3561.0,520,10X_P4_3,Bladder,,M,3-M-8,0.000000,0.157119,...,0.0,0,0,0.193333,Singlet,7,12,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
P8.MAA000487.3_10_M.1.1,Bladder,269238.0,637,NaN,Bladder,,M,3_10_M,0.004536,0.046559,...,0.0,1,1,0.121212,Singlet,2,1,1668602.0,MAA000487,Multiple
P9.B000610.3_56_F.1.1,Bladder,121666.0,522,NaN,Bladder,,F,3_56_F,0.008582,0.065607,...,0.0,4,4,0.139394,Singlet,3,2,865291.0,B000610,Multiple
P9.B002771.3_39_F.1.1,Bladder,187199.0,556,NaN,Bladder,,F,3_39_F,0.005361,0.043952,...,0.0,1,1,0.151515,Singlet,2,3,1056676.0,B002771,Multiple
P9.D042253.3_9_M.1.1,Bladder,109706.0,612,NaN,Bladder,,M,3_9_M,0.004746,0.015042,...,0.0,3,3,0.072727,Singlet,0,4,493818.0,D042253,Multiple


In [14]:
adata.obs['orig.ident'].unique
batch_key = 'orig.ident'

In [12]:
sc.pp.highly_variable_genes(adata, batch_key = 'orig.ident', subset=False)

In [15]:
sc.pp.scale(adata)
sc.pp.pca(adata, use_highly_variable=True) #True since we didnt subset
sce.pp.harmony_integrate(adata, key = batch_key)
sc.pp.neighbors(adata, use_rep = "X_pca_harmony")

2025-06-30 04:21:34,745 - harmonypy - INFO - Computing initial centroids with sklearn.KMeans...
2025-06-30 04:21:40,282 - harmonypy - INFO - sklearn.KMeans initialization complete.
2025-06-30 04:21:40,330 - harmonypy - INFO - Iteration 1 of 10
2025-06-30 04:21:41,958 - harmonypy - INFO - Iteration 2 of 10
2025-06-30 04:21:43,655 - harmonypy - INFO - Iteration 3 of 10
2025-06-30 04:21:45,318 - harmonypy - INFO - Iteration 4 of 10
2025-06-30 04:21:46,972 - harmonypy - INFO - Iteration 5 of 10
2025-06-30 04:21:48,833 - harmonypy - INFO - Iteration 6 of 10
2025-06-30 04:21:50,470 - harmonypy - INFO - Converged after 6 iterations


In [6]:
adata = sc.read_h5ad('../../../oscb/user_storage/Benchmarks/facs-Bladder_1751302627486/QC/integration/SEURAT_integration.h5ad')
adata

AnnData object with n_obs × n_vars = 3878 × 573
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCounts_RNA', 'nFeatures_RNA', 'nReads', 'plate.barcode', 'mouse.id', 'tissue', 'subtissue', 'FACS.selection', 'mouse.sex', 'percent.ercc', 'free_annotation', 'cell_ontology_class', 'percent.ribo', 'res.0.4', 'cluster.ids', 'cell_ontology_id', 'pct_counts_mt', 'pct_counts_rb', 'pct_counts_hb', 'percent.plat', 'RNA_snn_res.0.5', 'seurat_clusters', 'doublet_score', 'doublet_class', 'leiden', 'louvain', 'channel', 'previous_free_annotation', 'previous_cell_ontology_class', 'integrated_snn_res.0.8'
    var: 'name'
    uns: 'neighbors', 'umap'
    obsm: 'X_pca', 'X_umap', 'X_umap_3D'
    layers: 'scale.data'
    obsp: 'connectivities', 'distances'

In [7]:
adata.var

,name
Sprr2f,Sprr2f
Sprr2g,Sprr2g
Ccl2,Ccl2
Cxcl10,Cxcl10
Cxcl1,Cxcl1
...,...
Flnc,Flnc
Prss27,Prss27
E2f1,E2f1
Nell2,Nell2


In [8]:
sc.pp.highly_variable_genes(adata, n_top_genes=2000)

In [9]:
adata

AnnData object with n_obs × n_vars = 3878 × 573
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCounts_RNA', 'nFeatures_RNA', 'nReads', 'plate.barcode', 'mouse.id', 'tissue', 'subtissue', 'FACS.selection', 'mouse.sex', 'percent.ercc', 'free_annotation', 'cell_ontology_class', 'percent.ribo', 'res.0.4', 'cluster.ids', 'cell_ontology_id', 'pct_counts_mt', 'pct_counts_rb', 'pct_counts_hb', 'percent.plat', 'RNA_snn_res.0.5', 'seurat_clusters', 'doublet_score', 'doublet_class', 'leiden', 'louvain', 'channel', 'previous_free_annotation', 'previous_cell_ontology_class', 'integrated_snn_res.0.8'
    var: 'name', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'neighbors', 'umap', 'hvg'
    obsm: 'X_pca', 'X_umap', 'X_umap_3D'
    layers: 'scale.data'
    obsp: 'connectivities', 'distances'

In [11]:
adata.X.todense()

matrix([[ 2.4614373e-02,  0.0000000e+00, -1.1046639e-01, ...,
          0.0000000e+00,  4.4009313e-02, -1.2232100e-01],
        [ 2.7726073e-02,  2.5714863e-02,  1.4935477e+00, ...,
          7.3401652e-02,  0.0000000e+00,  4.3403778e-02],
        [ 2.6595723e-02,  0.0000000e+00,  1.2775791e-01, ...,
          1.3013720e-02, -3.0150579e-04, -2.5461438e-01],
        ...,
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
          0.0000000e+00,  2.6773479e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
          0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
          1.5437008e+00,  0.0000000e+00,  1.5437008e+00]], dtype=float32)